In [2]:
import zmq
import pandas as pd
import numpy as np
import datetime

context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://127.0.0.1:5555')
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

df = pd.DataFrame()
mom = 3
min_length = mom + 1

while True:
    data = socket.recv_string()
    t = datetime.datetime.now()
    sym , value = data.split()
    df = df.append(pd.DataFrame({sym: float(value)}, index=[t]))
    dr = df.resample('5s', label = 'right').last()
    dr['returns'] = np.log(dr/ dr.shift(1))
    if len(dr) > min_length:
        min_length += 1
        dr['momentum'] = np.sign(dr['returns'].rolling(mom).mean())
        print('\n' + '=' * 51)
        print('NEW SIGNAL | {}'.format(datetime.datetime.now()))
        print('=' * 51)
        print(dr.iloc[:-1].tail())
        if dr['momentum'].iloc[-2] == 1.0:
            print('\nLong Market Position')
        elif dr['momentum'].iloc[-2] == -1.0:
            print('\nShort Market Position')


NEW SIGNAL | 2021-07-01 20:10:05.164979
                         SYMBOL   returns  momentum
2021-07-01 20:09:50  100.706081       NaN       NaN
2021-07-01 20:09:55  100.575016 -0.001302       NaN
2021-07-01 20:10:00  100.083758 -0.004896       NaN
2021-07-01 20:10:05  100.744343  0.006579       1.0

Long Market Position

NEW SIGNAL | 2021-07-01 20:10:10.669017
                         SYMBOL   returns  momentum
2021-07-01 20:09:50  100.706081       NaN       NaN
2021-07-01 20:09:55  100.575016 -0.001302       NaN
2021-07-01 20:10:00  100.083758 -0.004896       NaN
2021-07-01 20:10:05  100.744343  0.006579       1.0
2021-07-01 20:10:10  100.262285 -0.004796      -1.0

Short Market Position

NEW SIGNAL | 2021-07-01 20:10:15.111331
                         SYMBOL   returns  momentum
2021-07-01 20:09:55  100.575016 -0.001302       NaN
2021-07-01 20:10:00  100.083758 -0.004896       NaN
2021-07-01 20:10:05  100.744343  0.006579       1.0
2021-07-01 20:10:10  100.262285 -0.004796      -1.0



NEW SIGNAL | 2021-07-01 20:11:35.016313
                         SYMBOL   returns  momentum
2021-07-01 20:11:15   99.947740  0.000118      -1.0
2021-07-01 20:11:20   99.039970 -0.009124      -1.0
2021-07-01 20:11:25  100.026473  0.009911       1.0
2021-07-01 20:11:30   99.268786 -0.007604      -1.0
2021-07-01 20:11:35   99.971561  0.007055       1.0

Long Market Position

NEW SIGNAL | 2021-07-01 20:11:41.417977
                         SYMBOL   returns  momentum
2021-07-01 20:11:20   99.039970 -0.009124      -1.0
2021-07-01 20:11:25  100.026473  0.009911       1.0
2021-07-01 20:11:30   99.268786 -0.007604      -1.0
2021-07-01 20:11:35   99.971561  0.007055       1.0
2021-07-01 20:11:40   99.508922 -0.004638      -1.0

Short Market Position


KeyboardInterrupt: 